
<h2 style="font-size:34px; font-family:Verdana" align="center">Linear Models </h2>


<a id='0'></a>

<a id='1'></a>

## 1. Naive Bayes

Classification is a fundamental issue in machine learning and data mining. In classification, the goal of a learning algorithm is to construct a classifier given a set of training examples with class labels. Typically, an example $E$ is represented by a tuple of attribute values ($x_1, x_2, … , x_n$), where $x_i$ is the value of attribute $X_i$. 

Let $C$ represent the classification variable, and let $c$ be the value of $C$. Here, we assume that there are only two classes: + (the positive class) or − (the negative class).

A classifier is a function that assigns a class label to an example. From the probability perspective, according to Rule, the probability of an example $E = (x_1, x_2, … , x_n)$ being class $c$ is

$$p(c|E) = {\frac{p(E|c)p(c)}{p(E)}}$$

$E$ is classified as the class $C = +$ if and only if
$$f_b(E) = {\frac{p(C = +|E)}{p(C = -|E)}}\geq1$$


where $f_b(E)$ is called a Bayesian classifier. Assume that all attributes are independent given the value of the class variable; that is,
$$p(E|c) = p(x_1, x_2, … , x_n|c) = \prod\limits_{i = 1}^n p(x_i|c)$$
the resulting classifier is then:
$$f_{nb}(E) = {\frac{p(C = +)}{p(C = -)}} \prod\limits_{i = 1}^n {\frac{p(x_i|c = +)}{p(x_i|c = -)}} $$
The function $f_{nb}(E)$ is called a naive Bayesian classifier, or simply naive Bayes (NB). Figure 1 shows an example of naive Bayes. In naive Bayes, each attribute node has no parent except the class node.


<img src='http://i.piccy.info/i9/e3f799752a613ded34b7d17feaf2eca4/1492605985/18456/1138938/NB5.jpg'/>

Naive Bayes is the simplest form of Bayesian network, in which all attributes are independent given the value of the class variable. This is called conditional independence. It is obvious that the conditional independence assumption is rarely true in most real-world applications. A straightforward approach to overcome the limitation of naive Bayes is to extend its structure to represent explicitly the dependencies among attributes

<a id='2'></a>

In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
# Import data for train  
train = pd.read_csv('../data/movie_reviews.csv', sep = ',')
train = train [1:50000]
train_data = train.text
train_labels = train.label

In [3]:
# Import data for test 
test = pd.read_csv('../data/test.csv', sep = ',')
test_data = test.text
test_labels = test.label

In [4]:
# Create validation dataset
#train_data, train_validation_data, train_labels, train_validation_labels = train_test_split(train.text, train.label, test_size=0.1, random_state=42, stratify=train.label)

In [5]:
import os
import numpy as np

import zipfile

from sklearn.model_selection import train_test_split

# data path initialization
BASE_DIR = '../'
TEXT_DATA_DIR = BASE_DIR + 'data/'
TEXT_DATA_FILE = "movie_reviews.csv"
HEADER = True

# parameters initialization
VALIDATION_SPLIT = 0.1
RANDOM_SEED = 42


def load_data():
    # function for loading data
    x = []
    y = []
    with open(os.path.join(TEXT_DATA_DIR, TEXT_DATA_FILE), "r", encoding="utf-8") as f:
        if HEADER:
            _ = next(f)
        for line in f:
            temp_y, temp_x = line.rstrip("\n").split(",", 1)
            x.append(temp_x.replace("'", ""))
            y.append(temp_y)
    return x, y

data, labels = load_data()
labels = np.asarray(labels, dtype='int8')

# spliting our original data on train and validation sets
data_train, data_val, labels_train, labels_val = \
    train_test_split(data, np.asarray(labels, dtype='int8'),
                     test_size=VALIDATION_SPLIT, random_state=RANDOM_SEED, stratify=labels)

In [6]:
# Take the last 22 words from each review in the train set
#train_data = train_data.str.split().apply(lambda x:  ' '.join(x for x in x[-22:]))

In [7]:
# List of stopwords
STOPWORDS = ['by','does', 'was', 'were', 'the', 'of', 'end', 'and', 'is']    

In [9]:
# Train NB
classifier = MultinomialNB(alpha=0.4)
pipeline = Pipeline([('vectorizer', CountVectorizer(binary=True,ngram_range=(1,3),stop_words=STOPWORDS)), ('classifier', classifier)])
model = pipeline.fit(X=data_train, y=labels_train)

In [10]:
# Validation
pred_test = model.predict(data_val)

print ("Accuracy :", metrics.accuracy_score(labels_val, pred_test))
print ("F1-score :", metrics.f1_score(labels_val, pred_test))

Accuracy : 0.823799226787
F1-score : 0.853739461518


In [11]:
# Test 
pred_test = model.predict(test_data)

print ("Accuracy :", metrics.accuracy_score(test_labels, pred_test))
print ("F1-score :", metrics.f1_score(test_labels, pred_test))

Accuracy : 0.799718574109
F1-score : 0.815805366232


# 2. Linear Models (SVM, Logistic Regression)

### SVM Model overview  


A Support Vector Machine (SVM) is a supervised machine learning algorithm.

SVMs are more commonly used in classification problems.

SVMs are based on the idea of finding a hyperplane that best divides a dataset into two classes.

<img src="../pictures/svm_intro.png" alt="logistic" style="width: 100%;"/>


** Support Vectors **

Support vectors are the data points nearest to the hyperplane, the points of a data set that, if removed, would alter the position of the dividing hyperplane. Because of this, they can be considered the critical elements of a data set.
What is a hyperplane?

 
As a simple example, for a classification task with only two features (like the image above), you can think of a hyperplane as a line that linearly separates and classifies a set of data.

Intuitively, the further from the hyperplane our data points lie, the more confident we are that they have been correctly classified. We therefore want our data points to be as far away from the hyperplane as possible, while still being on the correct side of it.

So when new testing data is added, whatever side of the hyperplane it lands will decide the class that we assign to it.


** How do we find the right hyperplane? **


Or, in other words, how do we best segregate the two classes within the data?

The distance between the hyperplane and the nearest data point from either set is known as the margin. The goal is to choose a hyperplane with the greatest possible margin between the hyperplane and any point within the training set, giving a greater chance of new data being classified correctly.

<img src="../pictures/svm_margins.png" alt="logistic" style="width: 70%;"/>


** But what happens when there is no clear hyperplane? **  

This is where it can get tricky. Data is rarely ever as clean as our simple example above.  
A dataset will often look more like the jumbled balls below which represent a linearly non separable dataset.  
In order to classify a dataset like the one above it’s necessary to move away from a 2d view of the data to a 3d view.  

Imagine that our two sets of colored balls above are sitting on a sheet and this sheet is lifted suddenly, launching the balls into the air. While the balls are up in the air, you use the sheet to separate them. This ‘lifting’ of the balls represents the mapping of data into a higher dimension. This is known as kernelling. 

<img src="../pictures/svm_kerneling.png" alt="logistic" style="width: 70%;"/>

Because we are now in three dimensions, our hyperplane can no longer be a line. It must now be a plane as shown in the example above. The idea is that the data will continue to be mapped into higher and higher dimensions until a hyperplane can be formed to segregate it.


** SVM Uses **
 
SVM is used for text classification tasks such as category assignment, detecting spam and sentiment analysis.  
It is also commonly used for image recognition challenges, performing particularly well in aspect-based recognition and color-based classification.  
SVM also plays a vital role in many areas of handwritten digit recognition, such as postal automation services.


### SVM Main parameters

** sklearn.svm.LinearSVC **

** C ** : float, default: 1.0  
    Inverse of regularization strength; must be a positive float.  
    Like in support vector machines, smaller values specify stronger regularization. 
    
** loss ** : string, ‘hinge’ or ‘squared_hinge’ (default=’squared_hinge’)  
    Specifies the loss function. ‘hinge’ is the standard SVM loss (used e.g. by the SVC class) while ‘squared_hinge’ is the square of the hinge loss.

** penalty **: string, ‘l1’ or ‘l2’ (default=’l2’)  
    Specifies the norm used in the penalization. The ‘l2’ penalty is the standard used in SVC. The ‘l1’ leads to coef_ vectors that are sparse, while ‘l2’ shrinks them close to 0

** tol **: float, optional (default=1e-4)
    Tolerance for stopping criteria.

In [12]:
pipeline_svm = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1, 3))),
                     ('clf_svm', LinearSVC(class_weight='balanced'))])

In [ ]:
pipeline_svm.steps[1]

In [ ]:
params_svm = dict(clf_svm__C=[100, 10, 1])
grid_search_svm = GridSearchCV(pipeline_svm, param_grid=params_svm, cv=5, scoring='accuracy')
%time grid_search_svm.fit(X=data_train, y=labels_train)

In [ ]:
print("Best parameters set found on development set:")
print()
print(grid_search_svm.best_params_)

In [12]:
%%time 
pipeline_svm = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                     ('clf_svm', LinearSVC(class_weight='balanced', C = 10))])
model = pipeline_svm.fit(X=data_train, y=labels_train)

CPU times: user 1min 45s, sys: 1.76 s, total: 1min 47s
Wall time: 1min 47s


In [13]:
# Validation
pred_test = model.predict(data_val)

print ("Accuracy :", metrics.accuracy_score(labels_val, pred_test))
print ("F1-score :", metrics.f1_score(labels_val, pred_test))

Accuracy : 0.841294803748
F1-score : 0.867039964866


In [14]:
pred_test = model.predict(test_data)

print ("Accuracy :", metrics.accuracy_score(test_labels, pred_test))
print ("F1-score :", metrics.f1_score(test_labels, pred_test))

Accuracy : 0.829549718574
F1-score : 0.838617994493


# 3. Logistic Regression  

### LR Model overview  

Logistic regression is a linear model for classification rather than regression.  
Logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier.  
In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function (sigmoid function).  

<img src="../pictures/sphx_glr_plot_logistic_001.png" alt="logistic" style="width: 70%;"/>

Decision boundary “separates” variable space into two decision regions.
Linear regression not advised for classification. First, it is sensitive to outliers/skewed data sets.

We will focus on ** sklearn.linear_model.LogisticRegression ** (liblinear)
This class implements regularized logistic regression using the "liblinear" library.


# Logistic function and SoftMax


### Logistic function
$$  
    F(x) = \frac{1}{1+e^{-(w_0 + x^T * w_1)}} 
$$

<br> where $w_0$ corresponds to intercept of the model, 
<br>$w_1$ - vector of model coefficients, 
<br>$x$ - feature vector 


### SoftMax
$$  
    F(x) = \frac{e^{ x^T * w_j}}{\sum_{k=1} e^{ x^T * w_k}} 
$$

<br> where $w_j$ model weights corresponding to class j,  
<br>$x$ - feature vector

### Logistic Regression Main params

** C ** : float, default: 1.0  
    Inverse of regularization strength; must be a positive float.  
    Like in support vector machines, smaller values specify stronger regularization.  

** class_weight **: dict or ‘balanced’, default: None
    Weights associated with classes in the form {class_label: weight}.  
    If not given, all classes are supposed to have weight one.

** solver **: {‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’}, default: ‘liblinear’
    Algorithm to use in the optimization problem.
        For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ is
            faster for large ones.
        For multiclass problems, only ‘newton-cg’, ‘sag’ and ‘lbfgs’ handle
            multinomial loss; ‘liblinear’ is limited to one-versus-rest schemes.

** tol **: float, default: 1e-4
    Tolerance for stopping criteria.


** n_jobs ** : int, default: 1
    Number of CPU cores used during the cross-validation loop. If given a value of -1, all cores are used.

In [15]:
pipeline_lr = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                     ('clf_lr', LogisticRegression())])

In [16]:
pipeline_lr.steps[1]

('clf_lr',
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False))

In [ ]:
params_lr = dict(clf_lr__C=[0.01, 0.1, 1, 10])
grid_search_lr = GridSearchCV(pipeline_lr, param_grid=params_lr, cv=5, scoring='accuracy')
%time grid_search_lr.fit(X=data_train, y=labels_train)

In [17]:
%%time
pipeline_lr = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                     ('clf_lr', LogisticRegression(C = 1))])
pipeline_lr.fit(X=data_train, y=labels_train)

CPU times: user 1min 55s, sys: 2.31 s, total: 1min 57s
Wall time: 1min 14s


In [18]:
# Validation
pred_test = model.predict(data_val)

print ("Accuracy :", metrics.accuracy_score(labels_val, pred_test))
print ("F1-score :", metrics.f1_score(labels_val, pred_test))

Accuracy : 0.841294803748
F1-score : 0.867039964866


In [19]:
pred_test = model.predict(test_data)

print ("Accuracy :", metrics.accuracy_score(test_labels, pred_test))
print ("F1-score :", metrics.f1_score(test_labels, pred_test))

Accuracy : 0.829549718574
F1-score : 0.838617994493


In [20]:
print(classification_report(test_labels, pred_test))

             precision    recall  f1-score   support

          0       0.87      0.77      0.82      5330
          1       0.80      0.89      0.84      5330

avg / total       0.83      0.83      0.83     10660



# LR vs SVM  

In practical classification tasks, linear logistic regression and linear SVMs often yield very similar results.  
** Logistic regression ** tries to maximize the conditional likelihoods of the training data, which makes it more prone to outliers than SVMs.  
** The SVMs ** mostly care about the points that are closest to the decision boundary (support vectors). 

On the other hand, logistic regression has the advantage that it is a simpler model that can be implemented more easily.   Furthermore, logistic regression models can be easily updated, which is attractive when working with streaming data.